<center><h1> Course Review Webscraping </h1></center>

## Import required library

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup # For HTML parsing
import requests # Website connections
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
import pandas as pd # For converting results to a dataframe and bar chart plots
import re # For regular expression
import matplotlib.pyplot as plt
%matplotlib inline

s = Service('Users\Hanme\Desktop\WeCloudData\week2_Python\chromedriver_win32\chromedriver.exe')

In [2]:
#Test: set up beautifulsoup
url = 'https://careerkarma.com/schools/weclouddata/'
result = requests.get(url)
soup = BeautifulSoup(result.content)

In [3]:
# test status of beautifulsoup
result.status_code

200

In [4]:
# setup driver
driver = webdriver.Chrome(service=s)

In [14]:
driver.get('https://careerkarma.com/schools/weclouddata/')
sleep(2)

In [6]:
#close pop-up
pop_up = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/div/div[2]/button')
pop_up.click()

In [7]:
#click sign in
sign_in = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div[1]/div/div/div[2]/div/div[2]/div/div[5]/div')
sign_in.click()
sleep(2)

In [8]:
#sign in
email = driver.find_element(By.XPATH, '//*[@id="layoutContentWrapper"]/div/div/div/div/div[1]/form/div/div[1]/span/input')
email.send_keys('hanmengqi77@outlook.com')
sleep(2)

password = driver.find_element(By.XPATH, '//*[@id="layoutContentWrapper"]/div/div/div/div/div[1]/form/div/div[2]/span/span/input')
password.send_keys('Data123456')
sleep(2)

driver.find_element(By.XPATH, '//*[@id="layoutContentWrapper"]/div/div/div/div/div[1]/form/button/span').click()
sleep(2)

In [15]:
# click review
driver.find_element(By.XPATH, '//*[@id="schoolTabs"]/div/a[3]').click()

In [27]:
scroll_pause_time = 2 # set pause time
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    
    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # Find the button element
    button = soup.find('button', {'class': 'sc-23911ef4-0 iYoOmN'})
    # Scroll to the button element using Selenium
    try:
        button_element = driver.find_element(By.XPATH, '//*[@id="reviews"]/div[2]/button')
        # Click the button
        button_element.click()
    except:
        #button not exist
        scroll_height = driver.execute_script("return document.body.scrollHeight;")
        break
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [30]:
#click the review icon
a = ActionChains(driver)
m = driver.find_element(By.XPATH, '//*[@id="reviews"]/div[2]/div[10]/div[1]/div/div[1]/div[2]/div[2]/span/div/div/span')
a.move_to_element(m).perform()

In [24]:
# total_review number
total_review = int(re.findall('(\d+)', driver.find_element(By.XPATH, '//*[@id="reviews"]/h2/div').text)[0])
total_review

70

In [35]:
# get sub-review for each link
total_review = int(re.findall('(\d+)', driver.find_element(By.XPATH, '//*[@id="reviews"]/h2/div').text)[0])

# sub-review
curriculum = []
instructor = []
Job_Assistance = []
Overall_Experience = []

for n in range(total_review):
    a = ActionChains(driver)
    m = driver.find_elements(By.XPATH, f'//*[@id="reviews"]/div[2]/div[{n+1}]/div[1]/div/div[1]/div/div[2]')
    for sub in m:
        a.move_to_element(sub).perform()
        sleep(2)

soup = BeautifulSoup(driver.page_source,'html.parser')

#class of starsdropdown
sub_review = soup.find_all('div', class_= 'sc-14dc08e0-0 gremWi')
#x is the number of total sub-review
x = len(sub_review)
print(x)

# for div in this sub_review, for loop for each div
for i in range(x):
    div = sub_review[i]
    
    # append curriculum
    if (i+1) % 4 == 1:
        a = 0
        for cur in div.find_all('img'):
            if cur.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                a += 1
        curriculum.append(a)

    # append instructor
    elif (i+1) % 4 == 2:
        b = 0
        for ins in div.find_all('img'):
            if ins.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                b += 1
        instructor.append(b)

    # append job assistance
    elif (i+1) % 4 == 3:
        c = 0
        for job in div.find_all('img'):
            if job.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                c += 1
        Job_Assistance.append(c)

    # append overall experience
    elif (i+1) % 4 == 0:
        d = 0
        for exp in div.find_all('img'):
            if exp.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                d += 1
        Overall_Experience.append(d)
    
print(len(curriculum))
print(len(instructor))
print(len(Job_Assistance))
print(len(Overall_Experience))

280
70
70
70
70


In [37]:
# overall review
#num = 0
for div in soup.find_all('div', class_= 'sc-14dc08e0-0 dZbmaa'):
    num += 1
    count = 0
    for i in div.find_all('img'):
        overall = i.get('src')
        if overall == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
            count += 1
    print(count)
#print(num)

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
4
5
5
5
5
5
5
5
5
5
4
4
5
3
5
3
5
5
70


In [36]:
# sub-review
curriculum = []
instructor = []
Job_Assistance = []
Overall_Experience = []

sub_review = soup.find_all('div', class_= 'sc-14dc08e0-0 gremWi')
#x is the number of total sub-review
x = len(sub_review)
print(x)

# for div in this sub_review, for loop for each div
for i in range(x):
    div = sub_review[i]
    # append curriculum
    if (i+1) % 4 == 1:
        a = 0
        for cur in div.find_all('img'):
            if cur.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                a += 1
        curriculum.append(a)

    # append instructor
    elif (i+1) % 4 == 2:
        b = 0
        for ins in div.find_all('img'):
            if ins.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                b += 1
        instructor.append(b)
    
    # append job assistance
    elif (i+1) % 4 == 3:
        c = 0
        for job in div.find_all('img'):
            if job.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                c += 1
        Job_Assistance.append(c)
    
    # append overall experience
    elif (i+1) % 4 == 0:
        d = 0
        for exp in div.find_all('img'):
            if exp.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                d += 1
        Overall_Experience.append(d)
print('cur',len(curriculum))
print('ins',len(instructor))
print('job',len(Job_Assistance))
print('exp',len(Overall_Experience))

280
cur 70
ins 70
job 70
exp 70


## Get all urls

In [14]:
List = ['weclouddata',
'practicum',
'metis',
'the-data-incubator',
'nyc-data-science-academy',
'data-science-retreat',
'brainstation',
'thinkful',
'springboard',
'general-assembly',
'le-wagon',
'university-of-toronto-school-of-continuing-studies-boot-camps',
'juno-college',
'lighthouse-labs']

In [15]:
links = []

for x in List:
    url = f'https://careerkarma.com/schools/{x}/'
    links.append(url)
print(links)

['https://careerkarma.com/schools/weclouddata/', 'https://careerkarma.com/schools/practicum/', 'https://careerkarma.com/schools/metis/', 'https://careerkarma.com/schools/the-data-incubator/', 'https://careerkarma.com/schools/nyc-data-science-academy/', 'https://careerkarma.com/schools/data-science-retreat/', 'https://careerkarma.com/schools/brainstation/', 'https://careerkarma.com/schools/thinkful/', 'https://careerkarma.com/schools/springboard/', 'https://careerkarma.com/schools/general-assembly/', 'https://careerkarma.com/schools/le-wagon/', 'https://careerkarma.com/schools/university-of-toronto-school-of-continuing-studies-boot-camps/', 'https://careerkarma.com/schools/juno-college/', 'https://careerkarma.com/schools/lighthouse-labs/']


In [16]:
for link in links:
    print(link)

https://careerkarma.com/schools/weclouddata/
https://careerkarma.com/schools/practicum/
https://careerkarma.com/schools/metis/
https://careerkarma.com/schools/the-data-incubator/
https://careerkarma.com/schools/nyc-data-science-academy/
https://careerkarma.com/schools/data-science-retreat/
https://careerkarma.com/schools/brainstation/
https://careerkarma.com/schools/thinkful/
https://careerkarma.com/schools/springboard/
https://careerkarma.com/schools/general-assembly/
https://careerkarma.com/schools/le-wagon/
https://careerkarma.com/schools/university-of-toronto-school-of-continuing-studies-boot-camps/
https://careerkarma.com/schools/juno-college/
https://careerkarma.com/schools/lighthouse-labs/


### Creat dictionary to store values

In [43]:
info = {
    'school':[],
    'overview':[],
    'courses':[],
    'financing options':[],
    'cities':[],
    'review title':[],
    'reviewer':[],
    'overall review':[],
    'Instructors':[],
    'Curriculum':[],
    'Job Assistance':[],
    'Overall Experience':[],
    'review':[],
    'pros':[],
    'cons':[]
}

In [ ]:
# find all data
school = driver.find_element(By.XPATH, '//*[@id="contentStart"]/div[1]/div[2]/th/div/h1').text
overview = driver.find_element(By.XPATH, '//*[@id="contentStart"]/div[1]/div[2]/div/div[1]/div[2]').text
courses = driver.find_element(By.XPATH, '//*[@id="layoutContentWrapper"]/div/div/div/div/div/div/div/div[1]/div[1]/div[2]/div/div[1]/div/div[2]/table/tr[1]/td/div').text
financing options = driver.find_element(By.XPATH, '//*[@id="layoutContentWrapper"]/div/div/div/div/div/div/div/div[1]/div[1]/div[2]/div/div[1]/div/div[2]/table/tr[2]/td').text
cities = driver.find_element(By.XPATH, '//*[@id="layoutContentWrapper"]/div/div/div/div/div/div/div/div[1]/div[1]/div[2]/div/div[1]/div/div[2]/table/tr[4]/td/div').text

review_title = soup.find_all('h2', class_='sc-4cffdd45-4 jzvPPu')
reviewer = soup.find_all('span', class_= 'sc-aefd771a-3 gEHSTK')

overall_review
curriculum
instructor
job assistance
overall experience

review = soup.find_all('div', class_='sc-4cffdd45-21 jeZAKJ')
pros = soup.find_all('div', class_='sc-4cffdd45-27 hwmILv')
cons = soup.find_all('div', class_='sc-4cffdd45-27 hwmILv')

In [38]:
# review title
for review_title in soup.find_all('h2', class_='sc-4cffdd45-4 jzvPPu'):
    info['review title'].append(review_title.get_text())
print(len(info['review title']))
info['review title']

70


['Full Time DS Bootcamp Review',
 'A good bootcamp! its worth to make the decision',
 'Best DS BootCamp in Canada',
 'Best DS in Canada',
 'Great program with helpful career guidance',
 'Great program with helpful career guidance',
 'Data Engineering boot camp - overall great experience',
 'Data Engineering boot camp - great experience',
 'Data Science Bootcamp',
 'Great Overall Experience',
 'Excellent All in One Program',
 'The best bootcamp very satisfied',
 'Great experience',
 'Best bootcamp for data science 😀',
 'Best DS program in Canada!!!',
 'Great experience',
 'Great way to enter into the world of Data Analytics',
 'A boost for your smooth career switch',
 'Experience with WeCloudData',
 'Experience with WeCloudData',
 'Great experience !',
 'Great Experience',
 'Great data analytics program',
 'One of the best decisions of my life! Brilliant faculty and great course structure!',
 'Very comprehensive bootcamp for BI/DA',
 'Best Decision I made',
 'Amazing bootcamp!!',
 'The 

In [39]:
for review in soup.find_all('div', class_='sc-4cffdd45-21 jeZAKJ'):
    info['review'].append(review.get_text())
print(len(info['review']))
info['review']

70


['Overall a great program especially for career switchers. WCD consolidated a lot of important instruction material in one place to build up technical and presentation skills for aspiring DA/DS professionals. Learning curve is steeper for those who come from non-technical backgrounds so the pre-bootcamp material is essential to review prior to the actual start of the bootcamp program. ',
 'I attended the data and business analytics bootcamp in early 2022 and now I have got my dream job! I am glad that I made this decision of joining the course. The tutor are nice and they are helpful even after graduated. The content included python, SQL, Tableau, power BI and excel, the videos are recoreded and can be reviewed unlimited time. Overall it was a good experience, I would definetely reccomand weclouddata to anyone.',
 "I arrived in Canada with a Bachelor's degree in Industrial engineering with no prior experience in analytics and a low English level. \n At WeCloudData, every instructor, TA

In [46]:
for num in range(len(soup.find_all('div', class_='sc-4cffdd45-27 hwmILv'))):
    comment = soup.find_all('div', class_='sc-4cffdd45-27 hwmILv')[num]
    if num % 2 == 0:
        info['pros'].append(comment.get_text())
    else:
        info['cons'].append(comment.get_text())
print(len(info['pros']))
print(info['pros'])
print(len(info['cons']))
print(info['cons'])

70
['- Knowledgeable and patient instructors and TAs\n- Relevant material taught throughout course\n- Hands-on approach to instruction helped expedite learning of material\n- Comprehensive advice for resume-building, job search tips, etc.', 'Great support, nice tutor, practical content', 'Really straight forward wth projects that helps you to practise your new skillset.', 'Really straight forward, with projects that help you to practise your new skills', 'Instructors/mentors/TAs are nice. My mentor Leno was very responsible. \nAlso they provided online programs', 'Instructors/mentors/TAs are nice. My mentor Leno was very responsible. \nThey provided online programs.', 'Great learning materials. \nValuable advice about best practices.\n', 'Great content and documentation. \nInstructors provided valuable advice in real work scenarios. I learned both theory and best practices.\nThe program also provided me a great career support.', 'Great program, team', "Client Projects\nNetworking Oppor

In [ ]:
# Initialize the line number
line_num = 0

# Set up for loop to iterate through all links
for link in links:
    
    driver.get(link)
    
    for i in range(len(total_review)):
    try:
        school = driver.find_element(By.XPATH, '//*[@id="contentStart"]/div[1]/div[2]/th/div/h1').text
        info['school'].append(school)
    except:
        info['school'].append('NA')
    
    try:
        overview = driver.find_element(By.XPATH, '//*[@id="contentStart"]/div[1]/div[2]/div/div[1]/div[2]').text
        info['overview'].append(overview)
    except:
        info['overview'].append('NA')
        
    try:
        course = driver.find_element(By.XPATH, '//*[@id="layoutContentWrapper"]/div/div/div/div/div/div/div/div[1]/div[1]/div[2]/div/div[1]/div/div[2]/table/tr[1]/td/div').text
        info['courses'].append(course)
    except:
        info['courses'].append('NA')
        
    try:
        financing_option = driver.find_element(By.XPATH, '//*[@id="layoutContentWrapper"]/div/div/div/div/div/div/div/div[1]/div[1]/div[2]/div/div[1]/div/div[2]/table/tr[2]/td').text
        info['financing options'].append('financing_option')
    except:
        info['financing options'].append('NA')
    
    try:
        city = driver.find_element(By.XPATH, '//*[@id="layoutContentWrapper"]/div/div/div/div/div/div/div/div[1]/div[1]/div[2]/div/div[1]/div/div[2]/table/tr[4]/td/div').text
        info['cities'].append(city)
    except:
        info['cities'].append('NA')
        
    try:
        review_title = soup.find_all('h2', class_='sc-4cffdd45-4 jzvPPu').get_text()
        info['review title'].append(review_title)
    except:
        info['review title'].append('NA')
    
    try:
        reviewer = soup.find_all('span', class_= 'sc-aefd771a-3 gEHSTK').get_text()
        info['reviewer'].append(reviewer)
    except:
        info['review title'].append('NA')

    for review_title in soup.find_all('h2', class_='sc-4cffdd45-4 jzvPPu'):
        info['review title'].append(review_title)
        
    for review in soup.find_all('div', class_='sc-4cffdd45-21 jeZAKJ'):
        info['review'].append(review.get_text())
        
    for num in range(len(soup.find_all('div', class_='sc-4cffdd45-27 hwmILv'))):
        comment = soup.find_all('div', class_='sc-4cffdd45-27 hwmILv')[num]
        if num % 2 == 0:
            info['pros'].append(comment.get_text())
        else:
            info['cons'].append(comment.get_text())
    
    
    

In [ ]:
# Set up for loop to iterate through all links
for link in links:
    driver.get(link)
    sleep(2)
    # click review
    driver.find_element(By.XPATH, '//*[@id="schoolTabs"]/div/a[3]').click()
    
    #scroll down the page to bottom to load all the reviews
    scroll_pause_time = 2 # set pause time
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1
    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  

        # Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        # Find the button element
        button = soup.find('button', {'class': 'sc-23911ef4-0 iYoOmN'})
        # Scroll to the button element using Selenium
        try:
            button_element = driver.find_element(By.XPATH, '//*[@id="reviews"]/div[2]/button')
            # Click the button
            button_element.click()
        except:
            #button not exist
            scroll_height = driver.execute_script("return document.body.scrollHeight;")
            break
    
    # number of total reviewer
    total_review = int(re.findall('(\d+)', driver.find_element(By.XPATH, '//*[@id="reviews"]/h2/div').text)[0])
    
    # append school
    for i in range(total_review):
        try:
            school = driver.find_element(By.XPATH, '//*[@id="contentStart"]/div[1]/div[2]/th/div/h1').text
            info['school'].append(school)
        except:
            info['school'].append('NA')
    print(len(info['school']))

    # append overview
    for i in range(total_review):
        try:
            overview = driver.find_element(By.XPATH, '//*[@id="contentStart"]/div[1]/div[2]/div/div[1]/div[2]').text
            info['overview'].append(overview)
        except:
            info['overview'].append('NA')
    print(len(info['overview']))
    
    # append review_title
    for review_title in soup.find_all('h2', class_='sc-4cffdd45-4 jzvPPu'):
        try:
            info['review title'].append(review_title)
        except:
            info['review title'].append('NA')
    print(len(info['review title']))
    
#     # append reviewer
#     for reviewer in soup.find_all('span', class_= 'sc-aefd771a-3 gEHSTK'):
#         try:
#             info['reviewer'].append(reviewer)
#         except:
#             info['reviewer'].append('NA')
#     print(len(info['reviewer']))
    
    # append review_star
    for div in soup.find_all('div', class_= 'sc-14dc08e0-0 dZbmaa'):
        star = 0
        for i in div.find_all('img'):
            overall = i.get('src')
            if overall == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                star += 1
        try:
            info['review star'].append(star)
        except:
            info['review star'].append('NA')
    print(len(info['review star']))
    
    # append Comments
    for comment in soup.find_all('div', class_='sc-4cffdd45-21 jeZAKJ'):
        try:
            info['Comments'].append(comment.get_text())
        except:
            info['Comments'].append('NA')
    print(len(info['Comments']))
    
    # append pros and cons
    for num in range(len(soup.find_all('div', class_='sc-4cffdd45-27 hwmILv'))):
        comment = soup.find_all('div', class_='sc-4cffdd45-27 hwmILv')[num]
        if num % 2 == 0:
            try:
                info['pros'].append(comment.get_text())
            except:
                info['pros'].append('NA')
        else:
            try:
                info['cons'].append(comment.get_text())
            except:
                info['cons'].append('NA')   
    print(len(info['pros']))
    print(len(info['cons']))
    
    #append stars in pop-up
    # perform the icon for each review
    for sub in driver.find_elements(By.XPATH, '//*[@id="reviews"]/div[2]/div/div[1]/div/div[1]/div/div[2]/span/div/div/span'):
        ActionChains(driver).move_to_element(sub).perform()
        sleep(2)

    soup = BeautifulSoup(driver.page_source,'html.parser')
    #class of starsdropdown
    sub_review = soup.find_all('div', class_= 'sc-14dc08e0-0 gremWi')
    #x is the number of total sub-review
    x = len(sub_review)
    print(x)

    # for div in this sub_review, for loop for each div
    for i in range(x):
        div = sub_review[i]

        # append curriculum
        if (i+1) % 4 == 1:
            a = 0
            for cur in div.find_all('img'):
                if cur.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                    a += 1
            try:
                info['Curriculum'].append(a)
            except:
                info['Curriculum'].append('NA')
            
        # append instructor
        elif (i+1) % 4 == 2:
            b = 0
            for ins in div.find_all('img'):
                if ins.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                    b += 1
            try:
                info['Instructors'].append(b)
            except:
                info['Instructors'].append('NA')
                
        # append job assistance
        elif (i+1) % 4 == 3:
            c = 0
            for job in div.find_all('img'):
                if job.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                    c += 1
            try:
                info['Job Assistance'].append(c)
            except:
                info['Job Assistance'].append('NA')
                
        # append overall experience
        elif (i+1) % 4 == 0:
            d = 0
            for exp in div.find_all('img'):
                if exp.get('src') == 'https://d1psgljc389n8q.cloudfront.net/web-illustrations/schools/bootcamps-star-checked.png':
                    d += 1
            try:
                info['Overall Experience'].append(d)
            except:
                info['Overall Experience'].append('NA')
    print(len(info['Curriculum']))
    print(len(info['Instructors']))
    print(len(info['Job Assistance']))
    print(len(info['Overall Experience']))
    
    # save data to dataframe
    df = pd.DataFrame(info)
    df.to_csv('course_review.csv',index=False,mode='a',header=False)
    
    # initialize dictionary for next iteration
    info = {
        'school':[],
        'overview':[],
        'review title':[],
        'review star':[],
        'Instructors':[],
        'Curriculum':[],
        'Job Assistance':[],
        'Overall Experience':[],
        'Comments':[],
        'pros':[],
        'cons':[]
    }
        
    sleep(2)

driver.close()
